In [ ]:
# Colab Cell 1: Install Libraries & Scrape Judicial Decision Links

# 1. Install necessary libraries
print("Installing required libraries...")
!pip install chromedriver_autoinstaller selenium beautifulsoup4 python-docx
# Add command to install chromium-browser
!apt-get update
!apt-get install -y chromium-browser

print("Libraries installed.\n")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Setup complete.


In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup Chrome driver
chromedriver_autoinstaller.install()

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.binary_location = "/usr/bin/chromium-browser"

driver = webdriver.Chrome(options=options)

# Result container
urls = []

# Pages 1 to 1626
for page in range(1, 1627):
    url = f"https://laws.moj.gov.sa/ar/JudicialDecisionsList/1?pageNumber={page}&pageSize=12&viewType=grid&courtTypes=1&sortingBy=2"
    print(f"🌐 Visiting page {page}")
    driver.get(url)

    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a.details-link"))
        )
    except Exception as e:
        print(f"❌ Page {page} timeout: {e}")
        continue

    links = driver.find_elements(By.CSS_SELECTOR, "a.details-link")
    for link in links:
        href = link.get_attribute("href")
        if href:
            urls.append(href)

    print(f"✅ Page {page}: {len(links)} links found.")

driver.quit()

# ✅ Summary
print(f"\n🎉 Total links collected: {len(urls)}")
for u in urls[:5]:
    print("🔗", u)

# --- New code to save links to a .txt file ---
output_filename = "collected_links.txt"
with open(output_filename, "w") as f:
    for link in urls:
        f.write(link + "\n")

print(f"All collected links have been saved to {output_filename}")

In [ ]:
# --- Step 0: Install nest_asyncio (needed for Colab to run async Playwright) ---
# Run this cell FIRST if you haven0t already, or after a fresh Colab session.
!pip install nest_asyncio

# --- Step 1: Install Playwright and its browsers (if you haven't already) ---
# Run this cell AFTER Step 0.
# If you've already run this in the current session, you can skip.
!pip install playwright
!playwright install
!pip install python-docx

In [14]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import re
import os

nest_asyncio.apply()

# --- DOCX Helper Functions ---

def apply_rtl_direction(paragraph):
    pPr = paragraph._element.get_or_add_pPr()
    for tag in ['w:bidi', 'w:textDirection', 'w:jc']:
        for child in pPr.findall(qn(tag)):
            pPr.remove(child)
    bidi = OxmlElement('w:bidi')
    bidi.set(qn('w:val'), '1')
    pPr.insert(0, bidi)
    text_direction = OxmlElement('w:textDirection')
    text_direction.set(qn('w:val'), 'rtl')
    pPr.insert(1, text_direction)

def add_line(doc, text):
    p = doc.add_paragraph(text)
    apply_rtl_direction(p)

# --- Main Async Function ---

async def main():
    input_links_filename = "collected_links.txt"
    fail_links_filename = "fail_links.txt"
    output_folder = "/content/drive/MyDrive/Temp/MOJ-D"
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(input_links_filename):
        print(f"'{input_links_filename}' not found. Creating a dummy file.")
        with open(input_links_filename, "w") as f:
            f.write("https://www.moj.gov.sa/ar/Judgements/Pages/ruling.aspx?requestid=6500123\n")

    with open(input_links_filename, "r") as f:
        urls = [line.strip() for line in f if line.strip()]

    if not urls:
        print("⚠️ No URLs found.")
        return

    print(f"✅ Loaded {len(urls)} URL(s). Output will be saved in: {output_folder}")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i, url in enumerate(urls):
            print(f"\n🔎 ({i+1}/{len(urls)}) Visiting: {url}")
            try:
                await page.goto(url, wait_until='domcontentloaded', timeout=30000)
                await page.wait_for_selector("div.text-rulling.selectable-text", timeout=20000)
            except PlaywrightTimeoutError:
                print(f"❌ Timeout loading: {url}")
                with open(fail_links_filename, "a") as f:
                    f.write(url + "\n")
                continue
            except Exception as e:
                print(f"❌ Failed to load page: {e}")
                with open(fail_links_filename, "a") as f:
                    f.write(url + "\n")
                continue

            async def safe_get_text(selector):
                try:
                    text = await page.locator(selector).first.text_content(timeout=5000)
                    return text.strip() if text else ""
                except Exception:
                    return ""

            try:
                case_number = await safe_get_text("xpath=//div[contains(text(),'القضية رقم')]")
                court = await safe_get_text("xpath=//div[contains(text(),'المحكمة')]")
                city = await safe_get_text("xpath=//div[contains(text(),'المدينة')]")
                judgment_number_raw = await safe_get_text("xpath=//div[contains(text(),'رقم الحكم')]")
                judgment_number = ''.join(filter(str.isdigit, judgment_number_raw)) or "UNKNOWN"
                date = await safe_get_text("xpath=//div[contains(text(),'التاريخ')]/span")

                ruling_text = ""
                locators = page.locator("div.text-rulling.selectable-text")
                count = await locators.count()
                for j in range(count):
                    html_content = await locators.nth(j).inner_html()
                    if html_content.strip():
                        ruling_text = html_content.replace("<br>", "\n").strip()
                        break
                if not ruling_text:
                    print("⚠️ No non-empty ruling text found.")
                    ruling_text = "N/A"

                doc = Document()
                style = doc.styles['Normal']
                style.font.name = 'Arial'
                rpr = style.element.rPr
                rpr.rFonts.set(qn('w:cs'), 'Arial')

                add_line(doc, f"القضية رقم: {case_number.split(':')[-1].strip()}" if case_number else "القضية رقم: N/A")
                add_line(doc, f"المحكمة: {court.split(':')[-1].strip()}" if court else "المحكمة: N/A")
                add_line(doc, f"المدينة: {city.split(':')[-1].strip()}" if city else "المدينة: N/A")
                add_line(doc, f"رقم الحكم: {judgment_number}")
                add_line(doc, f"التاريخ: {date}" if date else "التاريخ: N/A")
                add_line(doc, "\nنص الحكم الإبتدائي:")
                add_line(doc, ruling_text)

                clean_date = re.sub(r'\s+', '_', date.strip()) if date else "NoDate"
                clean_date = re.sub(r'[\\/:"*?<>|]+', '', clean_date)
                base_filename = f"{judgment_number}_{clean_date}"
                filename = os.path.join(output_folder, f"{base_filename}.docx")

                counter = 2
                while os.path.exists(filename):
                    filename = os.path.join(output_folder, f"{base_filename}_{counter}.docx")
                    counter += 1

                doc.save(filename)
                print(f"✅ Saved: {os.path.basename(filename)}")

            except Exception as e:
                print(f"❌ Error processing {url}: {e}")
                with open(fail_links_filename, "a") as f:
                    f.write(url + "\n")

        await browser.close()
        print("\n🎉 Done. Check the output folder for saved documents and 'fail_links.txt' for any failed URLs.")

# Run the async main function
if __name__ == "__main__":
    asyncio.run(main())


✅ Loaded 64 URL(s). Output will be saved in: /content/drive/MyDrive/Temp/MOJ-D

🔎 (1/64) Visiting: https://laws.moj.gov.sa/ar/JudicialDecisionsList/1/Xa0WF2wvjefNbdqE9Z3i0_sedMpgwfTXDyYE5flsNLv-GCplOTwLgpKGxPFv3H17
✅ Saved: 4530003115_٣٠_ذو_الحِجّة_١٤٤٤.docx

🔎 (2/64) Visiting: https://laws.moj.gov.sa/ar/JudicialDecisionsList/1/eQiH0HjI0CacudHuxT8rFfWSBOU_MW28IAGzBju-HD6n3ag8Zwln10dKTaIA7hIu
✅ Saved: 4431059550_٣٠_ذو_الحِجّة_١٤٤٤.docx

🔎 (3/64) Visiting: https://laws.moj.gov.sa/ar/JudicialDecisionsList/1/0ool46e65tAlTpctYbyaSpuZBeglLSSkcDBgOThhfZSi-MLt7OEqioJFW2hG7dVb
✅ Saved: 4431059550_٣٠_ذو_الحِجّة_١٤٤٤_2.docx

🔎 (4/64) Visiting: https://laws.moj.gov.sa/ar/JudicialDecisionsList/1/kiFXdB-wDz_iumPiIv8jzUyAOay4HoqGiYIN7VDmVsMeKAyA32doZlTFm2UfgNzv
✅ Saved: 4431069098_٣٠_ذو_الحِجّة_١٤٤٤.docx

🔎 (5/64) Visiting: https://laws.moj.gov.sa/ar/JudicialDecisionsList/1/au5VrzzV7_Flbm9JV3iWqZJ8o6rYnS7-nNHuhxI1EcpmPQPSwWc5GffcNDA8ASkb
✅ Saved: 4530002510_٣٠_ذو_الحِجّة_١٤٤٤.docx

🔎 (6/64) Visiting:

In [ ]:
!playwright install